In [ ]:
# ==============================================================================
# Step 0: ライブラリの準備
# ==============================================================================
!pip install -q gspread pandas pulp
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import networkx as nx
import pulp

# ==============================================================================
# Step 1: 認証とデータ読み込み
# ==============================================================================
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# ★ ここに自分のGoogleスプレッドシートのURLを貼り付けてください ★
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1tgnYbu__5ORGcRBgKQGaxeeOGJhgUE6QQ2MOA9_P36E/edit?usp=drive_link"
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ 認証完了")

try:
    workbook = gc.open_by_url(SPREADSHEET_URL)
    district_df = pd.DataFrame(workbook.worksheet("1-district").get_all_records())
    adjacency_df = pd.DataFrame(workbook.worksheet("2-adjacency").get_all_records())
    print("✅ データ読み込み完了")
    data_loaded_successfully = True
except Exception as e:
    print(f"🛑 データ読み込みエラー: {e}")
    data_loaded_successfully = False

# ==============================================================================
# Step 2: メイン処理
# ==============================================================================
if data_loaded_successfully:

    # --- パラメータ入力 ---
    print("\n--- パラメータ入力 ---")
    start_district_str = input("移動販売車の出発地点を入力してください (例: 松山一丁目): ")
    num_districts_str = input("移動販売車が巡回する地区の数を入力してください (スタート地点を含む, デフォルト: 6): ")

    # パラメータ設定 (空ならデフォルト値)
    # v_start: 移動販売車の出発地
    start_district = start_district_str if start_district_str else "松山一丁目"
    # K (巡回地区数)
    num_districts = int(num_districts_str) if num_districts_str else 6
    # H_max: 最大移動コスト (最大パス長)
    max_hops = num_districts - 1
    print(f"\n設定値: 出発地='{start_district}', 巡回地区数={num_districts} (→ 最大パス長={max_hops})")

    # --- データ準備 ---
    # グラフG=(V, E)の構築
    G = nx.from_pandas_edgelist(adjacency_df, 'district-1', 'district-2')
    # 集合V: 全地区のリスト
    districts = district_df['district'].tolist()
    # 定数p_i: 各地区の65歳以上人口
    pop65_dict = {row['district']: int(str(row['population-65']).replace(',', '')) for _, row in district_df.iterrows()}
    # 定数δ_i: 各地区の初期店舗有無
    has_store_initial = district_df.set_index('district')['store-exist'].apply(lambda x: str(x).lower() == 'yes').to_dict()

    if start_district not in districts:
        print(f"🛑 エラー: 出発地点 '{start_district}' が地区リストに存在しません。")
    else:
        # --- 3. 現状分析 ---
        print("\n--- 3. 現状分析 ---")
        # 「店舗空白地区の高齢者人口」を事前に計算
        initial_difficult_pop = {d: pop65_dict.get(d, 0) for d in districts if not has_store_initial.get(d, False)}
        total_initial_difficult_pop = sum(initial_difficult_pop.values())
        print(f"導入前の店舗空白地区の高齢者人口 (合計): {total_initial_difficult_pop:,.0f} 人")

        # --- 4. 最適化の実行 ---
        print("\n--- 4. 最適化の実行 ---")

        # 4-1. 目的関数係数の計算
        # 定数c_i: 地区iを訪問した場合のスコア（削減される困難人口）
        # c_i = (1 - δ_i) * p_i の実装
        coeffs = initial_difficult_pop.copy()

        # 4-2. 最適化モデルの構築
        model = pulp.LpProblem("Orienteering_Problem_Final", pulp.LpMaximize)

        # --- 決定変数の定義 ---
        # x_i: 地区iを巡回経路に含めるか(1)否(0)かを示すバイナリ変数
        x = pulp.LpVariable.dicts("Visit", districts, cat='Binary')
        # u_ij: 辺(i,j)を移動するか(1)否(0)かを示すバイナリ変数
        u = pulp.LpVariable.dicts("Move", list(G.edges()) + [(j, i) for i, j in G.edges()], cat='Binary')
        # t_i: 地区iの訪問順序を示す整数変数 (MTZ制約用)
        t = pulp.LpVariable.dicts("Order", districts, lowBound=0, cat='Integer')

        # --- 目的関数 (式5) ---
        # 訪問した地区のスコア(c_i)と訪問有無(x_i)の積の総和を最大化
        model += pulp.lpSum([coeffs.get(i, 0) * x[i] for i in districts]), "Total_Reduction"

        # --- 制約条件 ---
        # 制約(6): 経路の総移動コスト(パスの長さ)は、上限H_maxと等しい
        model += pulp.lpSum(u[edge] for edge in u) == max_hops, "Max_Hops_Constraint"
        # 制約(7): 訪問地区数は、パスの長さ + 1
        model += pulp.lpSum(x[i] for i in districts) == max_hops + 1, "Num_Visited_Constraint"

        # 制約(8): 出発地は必ず訪問する
        model += x[start_district] == 1, "Start_Must_Be_Visited"

        # 制約(9, 10): フロー保存則と連結性に関する制約
        for i in districts:
            # 地区i に入ってくるフロー(経路)の合計
            in_flow = pulp.lpSum(u.get((j, i), 0) for j in G.neighbors(i))
            # 地区i から出ていくフロー(経路)の合計
            out_flow = pulp.lpSum(u.get((i, j), 0) for j in G.neighbors(i))

            # 訪問しない地区(x_i=0)には、フローは出入りできない
            model += in_flow <= x[i]
            model += out_flow <= x[i]

            # 各地区の役割に応じたフローのバランスを定義
            if i == start_district:
                # 出発地: 出ていくフローが1多くなる (out - in = 1)
                model += out_flow - in_flow == 1
            else:
                # 中間地点: 入ってきたら必ず出ていく (in = out)
                # 終点: 入ってくるだけで出ていかない (in = 1, out = 0)
                # これらをまとめて表現する制約
                model += in_flow - out_flow >= 0
                model += in_flow <= 1

        # 制約(11-14): MTZ法による部分巡回路除去
        # M: 十分に大きな数
        M = len(districts) + 1
        # (11) スタート地点の訪問順は1番目
        model += t[start_district] == 1
        for i in districts:
            if i != start_district:
                # (12) 訪問しない地区(x_i=0)の訪問順は0になるようにする
                model += t[i] <= M * x[i]
                # (13) 訪問する地区(x_i=1)の訪問順は2番目以降
                model += t[i] >= 2 * x[i] - M * (1 - x[i])

        # (14) もしiからjへ移動する(u_ij=1)なら、jの訪問順はiより後でなければならない
        for i, j in u:
            if j != start_district:
                model += t[j] >= t[i] + 1 - M * (1 - u[(i,j)])

        # 求解
        model.solve()
        solver_status = pulp.LpStatus[model.status]

        # --- 5. 計算結果の表示 ---
        print("\n--- 5. 計算結果 ---")
        print(f"ソルバーのステータス: {solver_status}")

        if solver_status == 'Optimal':
            visited_districts = {d for d in districts if x[d].value() > 0.5}
            total_reduction = pulp.value(model.objective)
            final_difficult_pop = total_initial_difficult_pop - total_reduction

            # 最適化の結果から、訪問順序(t)に基づいて実際の経路を復元
            path_with_order = sorted([(d, t[d].value()) for d in visited_districts], key=lambda item: item[1])
            path = [d for d, order in path_with_order]

            print(f"\n【最適巡回経路 ({len(path)}地区, {len(path)-1}ホップ)】")
            print(" -> ".join(path))

            print("\n【指標サマリー】")
            print(f"導入前の店舗空白地区の高齢者人口: {total_initial_difficult_pop:15,.0f} 人")
            print(f"最適巡回による削減人口      : {total_reduction:15,.0f} 人")
            print("-" * 40)
            print(f"巡回後の店舗空白地区の高齢者人口: {final_difficult_pop:15,.0f} 人")
        else:
            print("\n最適解が見つかりませんでした。巡回地区数を増減させるなど、制約を調整してみてください。")

✅ 認証完了
✅ データ読み込み完了

--- パラメータ入力 ---
移動販売車の出発地点を入力してください (例: 松山一丁目): 竹丘三丁目
移動販売車が巡回する地区の数を入力してください (スタート地点を含む, デフォルト: 6): 8

設定値: 出発地='竹丘三丁目', 巡回地区数=8 (→ 最大パス長=7)

--- 3. 現状分析 ---
導入前の店舗空白地区の高齢者人口 (合計): 9,224 人

--- 4. 最適化の実行 ---

--- 5. 計算結果 ---
ソルバーのステータス: Optimal

【最適巡回経路 (8地区, 7ホップ)】
竹丘三丁目 -> 竹丘一丁目 -> 松山三丁目 -> 松山二丁目 -> 野塩一丁目 -> 野塩二丁目 -> 中里二丁目 -> 中里一丁目

【指標サマリー】
導入前の店舗空白地区の高齢者人口:           9,224 人
最適巡回による削減人口      :           3,995 人
----------------------------------------
巡回後の店舗空白地区の高齢者人口:           5,229 人
